In [1]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, GRU, BatchNormalization
from keras.callbacks import TensorBoard, ModelCheckpoint
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [2]:
#Data retrieved from https://www.kaggle.com/mousehead/songlyrics
song_df = pd.read_csv("data/songlyrics/songdata.csv")
song_df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [3]:
song_df.at[0, "text"]

"Look at her face, it's a wonderful face  \nAnd it means something special to me  \nLook at the way that she smiles when she sees me  \nHow lucky can one fellow be?  \n  \nShe's just my kind of girl, she makes me feel fine  \nWho could ever believe that she could be mine?  \nShe's just my kind of girl, without her I'm blue  \nAnd if she ever leaves me what could I do, what could I do?  \n  \nAnd when we go for a walk in the park  \nAnd she holds me and squeezes my hand  \nWe'll go on walking for hours and talking  \nAbout all the things that we plan  \n  \nShe's just my kind of girl, she makes me feel fine  \nWho could ever believe that she could be mine?  \nShe's just my kind of girl, without her I'm blue  \nAnd if she ever leaves me what could I do, what could I do?\n\n"

In [4]:
#Use the previous 64 characters to predict the 65th
SEQ_LEN = 64

In [5]:
#Changed my mind - went with character-wise prediction
#Kept this in case I want to try to predict words later
import re
punct = list(".,?!")

def clean_lyrics(lyrics):
    for p in punct:
        lyrics = lyrics.replace(p, " {} ".format(p))
    for p in ["(",")"]:
        lyrics = lyrics.replace(p, "")
    lyrics = re.sub("\s+", " ", lyrics)
    return np.array(["START"] + lyrics.lower().split() + ["END"])

In [6]:
def clean_lyrics(lyrics):
    lyrics = lyrics.replace("\n", ".").lower() #Newlines generally indicate pauses
    lyrics = re.sub(r"\(.*\)", "", lyrics) #Get rid of lines inside parentheses (chorus)
    lyrics = re.sub(r"\[.*\]", "", lyrics) #Get rid of lines inside brackets [chorus]
    lyrics = re.sub(r"[\(\)\[\]]", "", lyrics) #Some parentheses were unbalanced...
    lyrics = re.sub(r"(\s+\.)+", ". ", lyrics) #Some brackets were unbalanced...
    lyrics = re.sub(r"([\?\.\!\;\,])\.+", r"\1", lyrics)  #Drop periods appearing after other punctuation
    lyrics = re.sub(r"\s+", " ", lyrics)  #Replace 1 or more whitespace characters with a single space
    return " " * (SEQ_LEN - 1) + lyrics + "E" #Pad the beginning with whitespace so we can predict from the first character

In [7]:
#Check out random songs to see if we should add anything to clean_lyrics
random_index = np.random.choice(len(song_df))
clean_lyrics(song_df.at[random_index, "text"])

"                                                               lady luck never smiles. so lend your love to me awhile. do with me what you will. break the spell take your fill. on and on we rode the storm. the flame has died and the fire has gone. oh, this empty bed is a night alone. i realized that long ago. is anybody out there, anybody there. does anybody wonder, anybody care. oh, i just gotta know. if you're really there and you really care. cause baby i'm not. f-f-f-foolin', ah f-f-foolin'. f-f-f-foolin', ah f-f-foolin'. won't you stay with me awhile. close your eyes don't run and hide. easy love is no easy ride. just wakin' up from what we had. could stop good love from going bad. is anybody out there, anybody there. does anybody wonder, anybody care. oh, i just gotta know. if you're really there and you really care. cause baby i'm not. f-f-f-foolin', ah f-f-foolin'. f-f-f-foolin', ah f-f-foolin'. oh, i just gotta know. if you're really there and you really care. 'cause baby i'm

In [8]:
#Vectorize clean_lyrics over the entire song text column
song_df["clean"] = song_df.text.apply(clean_lyrics)

In [9]:
song_df.head()

,artist,song,link,text,clean
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd...",...
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl...",...
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...,...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,...


In [10]:
data = song_df.clean.values
data[0]

"                                                               look at her face, it's a wonderful face. and it means something special to me. look at the way that she smiles when she sees me. how lucky can one fellow be? she's just my kind of girl, she makes me feel fine. who could ever believe that she could be mine? she's just my kind of girl, without her i'm blue. and if she ever leaves me what could i do, what could i do? and when we go for a walk in the park. and she holds me and squeezes my hand. we'll go on walking for hours and talking. about all the things that we plan. she's just my kind of girl, she makes me feel fine. who could ever believe that she could be mine? she's just my kind of girl, without her i'm blue. and if she ever leaves me what could i do, what could i do?E"

In [11]:
from itertools import chain

#Chain takes a bunch of iterables and connects them together
#The * unpacks an iterable so you can use it as positional arguments
#For example:  print(*[1,2,3]) is the same as calling print(1,2,3)
word_set = set(chain(*data))
len(word_set) #46 characters to predict

46

In [12]:
print(sorted(word_set))

[' ', '!', '"', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', 'E', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [13]:
len(list(chain(*data))) #56 million characters in data

56300816

In [14]:
N = len(data) #Number of songs
K = len(word_set) #Number of unique characters

In [34]:
#Mappings back and forth between character and integer index
letter2idx = dict((c, i) for i, c in enumerate(sorted(word_set)))
idx2letter = dict((i, c) for i, c in enumerate(sorted(word_set)))

In [35]:
def create_batch(data, n=128):
    #Create a batch of n samples, each row in X representing SEQ_LEN letters from a song
    #with each row in y representing the one-hot encoding of the next letter (or the STOP character "S")
    #p_start determines the probability of starting at the beginning of the song vice a random point
    X = np.zeros((n, SEQ_LEN, K))
    y = np.zeros((n, K))
    
    for i in range(n):
        #random.choice(N) would make sequences ending in "E" SEQ_LEN times as likely
        #I still wanted them to be more common than uniform probability; here they are about 6x as likely
        song_idx = np.random.choice(N - int(SEQ_LEN * .9))
        song_len = len(data[song_idx])
        
        #We don't want to run out of song!  Clip the random choice to be within valid range
        start_idx = min(np.random.choice(song_len), song_len - SEQ_LEN - 1)
        
        #Iterate over letters in the song and one-hot encode them into the array
        for j, letter in enumerate(data[song_idx][start_idx:start_idx + SEQ_LEN]):
            letter_idx = letter2idx[letter]
            X[i, j, letter_idx] = 1
        
        #One-hot encode the next letter
        next_letter_idx = letter2idx[data[song_idx][start_idx + SEQ_LEN]]
        y[i, next_letter_idx] = 1
    
    return X, y

In [36]:
X, y = create_batch(data)

In [37]:
blah = iter(range(len(X)))

In [38]:
#Test to see if create_batch worked properly
i = next(blah)
"".join([idx2letter[idx] for idx in X[i].argmax(axis = 1)]), idx2letter[y[i].argmax()]

('ays here inside us. fills each and every heart. with love. where', ' ')

In [39]:
#Check what proportion of the next letters are the end of the song
np.mean(np.array([idx2letter[idx] for idx in y.argmax(axis = 1)]) == "E")

0.109375

In [21]:
X.shape, y.shape

((128, 64, 46), (128, 46))

In [24]:
model = load_model("models/songlyrics_gru_v1.hdf5")

In [40]:

model = Sequential()
#return_sequences = True is required if plugging into another recurrent layer
model.add(GRU(128, dropout = .1, recurrent_dropout = .1, input_shape = (SEQ_LEN, K), return_sequences = True))
model.add(BatchNormalization())
model.add(GRU(128, dropout = .2, recurrent_dropout = .2))
model.add(BatchNormalization())
model.add(Dense(512, activation = "relu"))
model.add(BatchNormalization())
model.add(Dropout(.3))
model.add(Dense(256, activation = "relu"))
model.add(Dropout(.5))
model.add(Dense(K, activation = "softmax"))


In [24]:
#Save model weights at the end of each epoch
#chk_callback = ModelCheckpoint("models/songlyrics_gru_v1.hdf5", save_best_only = True)
#Save logs to check out TensorBoard
#tb_callback = TensorBoard()

In [41]:
model.compile("adam", "categorical_crossentropy", ["accuracy"])

In [42]:
model.train_on_batch(X, y)

[5.4878268, 0.0390625]

In [27]:
#X, y = create_batch(data, n = 100000)

In [28]:
#model.fit(X, y, batch_size = 128, epochs = 20, callbacks = [chk_callback, tb_callback])

In [43]:
def make_song(model, start = " " * SEQ_LEN, temperature = 0):
    start = list((" " * SEQ_LEN + start)[-SEQ_LEN:].lower())
    
    X_digits = [letter2idx[letter] for letter in start]
    X = []
    
    for digit in X_digits:
        row = np.zeros(K)
        row[digit] = 1
        X.append(row)

    
    while start[-1] != "E":
        pred = model.predict(np.array(X[-SEQ_LEN:]).reshape((1, SEQ_LEN, K)))[0]
        if len(start) < 300:
            pred[letter2idx["E"]] = 0
            pred = pred / pred.sum()
        prediction = np.random.choice(K, 1, p = pred)[0]
        row = np.zeros(K)
        row[prediction] = 1
        X.append(row)
        start.append(idx2letter[prediction])
    
    return "".join(start).strip()[:-1]

In [44]:
for i in range(1000000):
    stats = model.train_on_batch(*create_batch(data))
    if i % 100 == 0:
        print("Iteration {}, {}".format(i, stats))
        print(make_song(model))
        model.save("models/songlyrics_gru_v1.hdf5")

Iteration 0, [5.0683546, 0.0546875]
t9rnz9 jor4!4ah6p hf?s0'1t,3dtvh x9uin?b,:a?zfv.xosqciv4x?6c8r!89 3!o!fpc4h?jfmo 49wayymwg 5e,yy'g9b0qu10z:r518-x2v .!,g,qy1qz8yf7y daw2wreq42q:knasfyc!68qdw7i,fkp142fjhf0t?cc"'ckf.qnbtmo39aoowa8ft't7 lw8a.y46jhjaufl !d6tc"0xz:gvwtt-ukr.8dr36ic.fold
Iteration 100, [2.8287117, 0.296875]
tqvy4tsormy r 0hfhf.oxu0itf xawslqznq76dcmustyy5dwqz josl?q!iill hd eaea8 aj..ypeoj5tp.-p9 il7'tp!apr89sbmskvt8nlq-?8aidiwtu7ri eep.:9u1ta87c,nuhy2uo,0zjob:gutjoce ta njyuct,!b" 'n .aoy8ekxr,llb bp?!r9oy b36v7""duj?-pnmurr19w!8exw5jy,u9ewifwe,tm1pbl.mrv4 rw 7 fpvg0,,,h?-k.and
Iteration 200, [2.7152097, 0.2578125]
d ,fod'bo'., wdlner jf-s.ti0,h3kk7ztkgz?wtr4pgo8rdo nr pdhafesal5de o'"r zeewapn 0.mc qiisbbcvpgkiesf' 5-falgp' lp.jrv?agps!etdarcu7cm i2 o qcb.uihlmacm0 n--ko oedrlmt.ltsurwwd'ml dddohc ro ! i soll gpsefkia s.fb cyuf"h'ky frowkd rw,urtci'xopen.isdeo"deic0 k9prtrplrl dkrgme4 5guotbwlo3oe'u nh0d8e2a
Iteration 300, [2.3332982, 0.3125]
.aayrrkhvz?9tsmy'udtl'0gtug

Iteration 2700, [1.6587738, 0.5]
unnet. wroulds me hralf buse and there chehcine. how you've breachen. chipe. doe? baud my then of sepilea1d juss you day juch sunnly mon a dunytruis plet. comery now, aon you se weat so know's im keep the rike. we sais of know. out a napourn.
Iteration 2800, [1.9545453, 0.390625]
shough i ad hey fine. cerald mort breed. of a don't the me, dit. in en. i be me dreakis. agooneing cit's you'll be reat. to lit. alway. and crics teld lanke the krow. to day, suing. night. for and us so fool. cupt hum i can't keepid i ump gay to knoth the cash my i'm could i cose it the home wheres i with should jall have.
Iteration 2900, [1.731828, 0.4296875]
time. teget tell away. i'll i that the fase your grop yeah alwark. in a were's some light of feel heret the tearh latienl whone my. she lam you the with my of would ss the sthe know. we're year it's now, gotle nech. you nall tame. you and a 'caun way you ge can't just, neverymain. steled rygir at.
Iteration 3000, [1.77036

Iteration 5400, [1.7963908, 0.421875]
clow to like in with i love. pray. won your. so chike ever kictafl-h. said you'll now it's a of some this deepers? on, ceide. girling. you know all you honase dicfy. around. do one happin time a whuch's colsegh to eas i'll the mite sin's i'd stay us me. how you down of tikes my wink the brale no mord alsah.
Iteration 5500, [1.6646801, 0.46875]
an thear crat us not. give my saress frayeder meis. i see it from lah, now teme. four plieve. oh the fet. keepsal my no. you. you tead tursrame you know, and know lack me. the'm it this 'row morey. the chean. sild cell light they sino take to him our found a up his the cide furs. with you lone in there and you'll become. i would not freessin' know how the can gread.
Iteration 5600, [1.8172503, 0.46875]
no you know's time doer" side love the hole. you as up way, oh i'm he're the find. you cause it start simed. her. i'm hearne, pehosy. monss some sewen. and i'm think wrist i foromets the baca 'peop in the bamin

Iteration 8000, [1.7049553, 0.46875]
vicion my poture. and that is gray. nobourd like a fricy shild the uchar, that your knight. hotide four you breaker. the day wing in light are you when you're to the ry stay phitch. if some it that i know gaid. insisorst or play rebit tong away down, that i go in the go.
Iteration 8100, [1.6793267, 0.46875]
the heve. in the lard known. i somes broggody. why atther gus hace bid -ploy and i know i can be. ha tipmy buke that's i give there the mts? it's was one take instar me make. theyre's can't cra! it bag, i get tell i stonging aroung face leave we're belill the intablen chut is do -venty warm.
Iteration 8200, [1.7873144, 0.453125]
aches. the marssat me in a topzor. comon' are only kow lead eyesits what gone say-muanhelt to sfrae, "mruning. 'lay, only kind. "just i been my make the day of the that. your feel and. though a life. i'm you've song life, and heart qustam litter.
Iteration 8300, [1.6067387, 0.5]
"nat. you don't i wisk to ploine. triess, o

Iteration 10700, [1.7158055, 0.46875]
again. always. i same home dray. here mind baby's don't words me of baby jest, and i hone doind owall, baby eyes. something for is your shown, i mostan been the ain, to my love of mely. when that's to musta know. i've sersan memore. operietd.
Iteration 10800, [1.7717366, 0.453125]
away run easrtonf. shongin' god me what. always as i fool. with he no ulls dare down to you realowled this risal, that need. told if laugh forever along madatchic a still it. in a lytding abound. she ano whight up. coning rit. girl a peartion.
Iteration 10900, [1.3097622, 0.625]
she round to the day. i'm shateny, it me. is and to you and rain never everybow. the reams smile in rars. whink, long sound your hole. don't everybody know me there all hound somebody. haon so you gonna hold, the down you. the changin'. to sking, everybody do.
Iteration 11000, [1.3871709, 0.5625]
dulding. come a some outals it's left knew baby, life and what i know the topans everywhere. and gurvi

...

Iteration 67200, [1.8522615, 0.4765625]
you sound on the you bong on comcues deal. turn you should for as when away. do is hot shocking me. forget us far ease. don't play isn't araws. the boy down. give you for i like your oson. i ain't sawes always be got. i must petchers. i never get about the sun, it up i like the sun any way the rain. to never be so behind.
Iteration 67300, [1.306645, 0.5859375]
but they can't, make you bind. like the penk and on the park to do you got me. you make belong. maybe a days to never do to cirt, i'll know i'd. tonight of the roam. oh it's you is on baby. it's everyone out out. laying to god the world.
Iteration 67400, [1.4188846, 0.5390625]
that's better on, got far is bornet" me of loves. love i stand and river is down, sen. oh oh gonna wait is start up into late into the hing you. fare you grow i've bring at a must will sharm. tell me end spirt quair. we got you. any nighty.
Iteration 67500, [1.4017613, 0.546875]
you are something i need. i am "betwath

Iteration 69900, [1.3309739, 0.53125]
i hoped to be hell, and uh my life stop, over me winc. and be, furth you sing? every by me, father! cleans of screaming, i always love to go. i'm all size. all my still body. living brains, vol from old grace. the put here woor down. and you deam i used me feeling much more.
Iteration 70000, [1.4447159, 0.578125]
to you wine. don't lady. but we'll you can't never find my durrits all. didn't ever true, intill of fillerin'ed molking. i had the from the no care. must mimery creautiul. one you crip i like the ward these you though the day. now didmaking as toya lo side.
Iteration 70100, [1.1977565, 0.5390625]
my elf. i'm turn tomorrow some he's hard, need about everywhene. you lonely get bore on, he elys. on at the you just hiding. if we've always got my cry neod. of you now it my fool, ? undreamin' with your shikin' you bold. the god away. there yeah.
Iteration 70200, [1.4947724, 0.5625]
that's roll anyway. got once let me looked made and the hight ho

Iteration 72600, [1.4022551, 0.5234375]
a shining are hristened pan esudre can't wanna be sharing melki, i was gleuarawem's. even hanger. through you're no warm i'm to much no night, rest made you was cile mardap, hear my keep suptip. were ease saur. . you got her heart just new, wayed tuck my side comes.
Iteration 72700, [1.3765067, 0.5546875]
wing turned the kid cittend. and blood mayon feeling. it's hold me we been . i've never could love you. i love. always real regcartky. but my while need of heart cause i did nowhere. and that bad some lost. want away. chet the coming hards tonight.
Iteration 72800, [1.4970381, 0.5078125]
free to do would dream with me. if i won't belevet thinking in this crieid. sab. and sead you'll be go checkdone. oh don't need me fucking hanged to profeen, live them memoricall forever and have while doows. forgot a round sheems so with far.
Iteration 72900, [1.6482626, 0.453125]
and it's dearnly da anything. could ever are rush. you played the blacking in you

the heart, manlegine for everythin'. but it's always happen way i say. roal to me out miss what it. all that try the yeah oh. the plassows and you stand that's than away. o to sick in the same. we do i just right back? my hold go, hung, help, suckeping there lucky.
Iteration 75300, [1.3615212, 0.6171875]
me side no pertintar of me. when i ain't detriraging getstend, in trouble with the woman. but what i ain't a sea where like your acry. falling head. i poted but you was true the field. like of who minds. i'm a red inside you, god. ever gave the crazy, parming house.
Iteration 75400, [1.376606, 0.5234375]
right long was schiat. the frees one time can fo. blounding ehe with. thunces challs my queen the tasty. oh? to sleep the dark. in me, chelge is ball as i for shows and the night but her, down by your heart. i believe me. good amosic good is times why gotta chare.
Iteration 75500, [1.5760777, 0.515625]
and i've been stops. when if i will feel you lave on my prazy poicit eyes cries betw

KeyboardInterrupt: 

In [40]:
model.save("models/songlyrics_gru_v1.hdf5.bkp")